In [1]:
! pip install timm -q

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import os
import torch
from fastai import *
from fastai.basics import *
from fastai.callback.all import *
from fastai.vision import *
from fastai.vision.all import *
from fastai.metrics import error_rate, accuracy
import timm
from timm.models import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import random
import PIL
from timm.data import ImageDataset
from sklearn.metrics import classification_report

In [5]:
timm.list_models('*convnext*in22k')

['convnext_base_in22k',
 'convnext_large_in22k',
 'convnext_small_in22k',
 'convnext_tiny_in22k',
 'convnext_xlarge_in22k']

In [6]:
dls = ImageDataLoaders.from_folder('../input/clocks-data/my_data', train='train', valid='valid', 
                                   seed=42, bs = 32, item_tfms=Resize(128, method='squish'),
                                   batch_tfms=aug_transforms(size=128, min_scale=0.75),
                                   device=torch.device('cuda'))

In [7]:
#название базовой модели
model_to_test_1 = vision_learner(dls, 'convnext_xlarge_in22k', model_dir='', metrics=error_rate, pretrained=False)
model_to_test_1 = model_to_test_1.load('epochs_40_convnext_xlarge_in22k', strict=False)

In [8]:
import gc
_ = gc.collect()

In [9]:
model_to_test_1.fine_tune(1, 1e-2)
#еще одна эпоха тренировки на данных (из fast ai best practices)

epoch,train_loss,valid_loss,error_rate,time
0,0.064274,0.190297,0.030556,05:08


epoch,train_loss,valid_loss,error_rate,time
0,0.053733,0.126209,0.027083,10:24


In [10]:
def do_test(dls, model_to_test, path):
    my_tst_files = get_image_files(path).sorted()
    my_tst_dl = dls.test_dl(my_tst_files)
    probs,_,idxs = model_to_test.get_preds(dl = my_tst_dl, with_decoded = True)
    mapping = dict(enumerate(dls.vocab))
    reverse_mapping=dict(zip(dls.vocab,list(range(len(dls.vocab)))))
    results = pd.Series(idxs.numpy(),name="idxs").map(mapping)
    testY = []
    for item in my_tst_files:
        testY += [reverse_mapping[item.parent.name]]
    r1 = pd.Series(idxs.numpy(),name="idxs").map(mapping)
    r2 = pd.Series(np.array(testY),name="idxs").map(mapping)
    
    return r1, r2, classification_report(testY, idxs, digits=4)

In [36]:
r1, r2, cr = do_test(dls, model_to_test_1, '../input/my-clocks')

In [37]:
print(cr)

              precision    recall  f1-score   support

           1     1.0000    1.0000    1.0000         1
           6     0.0000    0.0000    0.0000         0
          33     1.0000    1.0000    1.0000         1
          36     0.0000    0.0000    0.0000         0
          46     0.0000    0.0000    0.0000         1
          47     0.0000    0.0000    0.0000         0
          48     1.0000    1.0000    1.0000         1
          50     0.0000    0.0000    0.0000         0
          53     0.0000    0.0000    0.0000         0
          57     1.0000    0.5000    0.6667         2
          60     0.0000    0.0000    0.0000         3
          61     1.0000    1.0000    1.0000         1
          62     1.0000    1.0000    1.0000         1
          66     0.0000    0.0000    0.0000         0
          68     1.0000    1.0000    1.0000         1
          74     1.0000    1.0000    1.0000         1
          83     0.3333    1.0000    0.5000         1
          88     0.0000    

In [38]:
print('Что получено','Что должно быть')
k = 0
all = 0
for t in zip(r1, r2):
    print('{}\t\t{}'.format(t[0], t[1]))
    if t[0] == t[1]:
        k += 1
        print('        верно')
    elif abs(int(t[0].split('-')[0]) * 60 + int(t[0].split('-')[1]) - int(t[1].split('-')[0]) * 60 - int(t[1].split('-')[0])) <= 30:
        print('     почти верно')
        k += 1
    else:
        if t[0].split('-')[0] == t[1].split('-')[0]:
            print('     верен час')
            k += 0.5
        if t[0].split('-')[1] == t[1].split('-')[1]:
            print('     верны минуты')
            k += 0.5
        else:
            print()
    all += 1



Что получено Что должно быть
1-05		1-05
        верно
11-45		11-45
        верно
4-55		12-50

2-00		2-00
        верно
6-00		2-45

2-45		2-45
        верно
3-30		3-00
     почти верно
12-00		3-00
     верны минуты
1-30		3-00

3-05		3-05
        верно
3-10		3-10
        верно
3-40		3-40
        верно
4-10		4-10
        верно
4-55		4-55
        верно
2-10		5-20

5-25		5-25
        верно
4-55		5-50
     почти верно
8-15		5-55

5-55		5-55
        верно
3-30		6-05

12-55		6-55
     верны минуты
5-35		8-35
     верны минуты
6-05		8-55

6-05		9-05
     верны минуты
9-15		9-10
     почти верно
2-25		9-25
     верны минуты
9-35		9-35
        верно


In [39]:
print('Общая оценка: {:.2f}' .format(k / all * 100))

Общая оценка: 64.81
